# Zagrava interview

First we import libs and get parameters from config and strategy files

strategy.yaml file will be our minimalistic interface in which the user can specify the conditions in which he or she will buy or sell. 

In [30]:
import os
import sys
import time
import pandas as pd
import ccxt
from ccxt.base.decimal_to_precision import ROUND_UP
import yaml

# get strategy parameters
with open("strategy.yaml", "r") as strategy:
    try:
        strat = yaml.safe_load(strategy)
    except yaml.YAMLError as exc:
        print(exc)

# get config parameters
with open("config.yaml", "r") as config:
    try:
        cfg = yaml.safe_load(config)
    except yaml.YAMLError as exc:
        print(exc)

msec = 1000
minute = 60 * msec
hold = 30

symbol = cfg['symbol']
windowSize = strat['windowSize']
# add windowSize to timeLimit to calculate ewma properly for given timeLimit
timeLimit = cfg['timeLimit'] + windowSize
timeframe = cfg['timeframe']
comissionRate = cfg['comissionRate']


Now let's get candles from binance. It will be our data collection unit that would gather 1 minute candles in the specified range.

In [31]:
# used some code from ccxt github: https://github.com/ccxt/ccxt/blob/master/examples/py/fetch-ohlcv-on-new-candle.py
exchange = ccxt.binance({
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future',  # futures contracts
    },
})

interval = exchange.parse_timeframe(timeframe) * 1000

try:
    since = exchange.round_timeframe(
        timeframe, exchange.milliseconds(), ROUND_UP) - (timeLimit * interval)
    ohlcv = exchange.fetch_ohlcv(symbol.replace(
        "/", ""), timeframe, since=since, limit=timeLimit)
except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:
    print('Got an error', type(error).__name__, error.args)

# leave only updatetime and close price columns and calculate simple moving average
candlesDf = pd.DataFrame(
    ohlcv, columns=['updatetime', 'open', 'high', 'low', 'close', 'volume'])
candlesDf = candlesDf[['updatetime', 'close']]
candlesDf['ma'] = candlesDf['close'].rolling(windowSize).mean()
# cut first windowSize rows to calculate ewma properly
candlesDf = candlesDf[windowSize:].reset_index(drop=True)

candlesDf


,updatetime,close,ma
0,1666958100000,1498.25,1498.955500
1,1666958160000,1498.78,1498.930000
2,1666958220000,1498.26,1498.869167
3,1666958280000,1498.24,1498.846167
4,1666958340000,1497.36,1498.799000
...,...,...,...
295,1666975800000,1552.40,1545.299833
296,1666975860000,1553.00,1545.595667
297,1666975920000,1552.79,1545.872500
298,1666975980000,1563.57,1546.365000


Now we have dataframe with close prices and moving everage for specified in config time range

Now let's code our processing unit that would, given the collected data and user strategy, would calculate PnL (realized + unrealized) during each 1min period. And don't forget about comission.

Function get_pnlDf will return two dataframes:<br /> 1) tradesDf wich we will use mostly for troubleshouting and checking that our strategy works as we intended<br /> 2) pndDf with pnl data for each candle

In [32]:
# pnl calculation algorithm was taken from:
# https://www.tradingtechnologies.com/xtrader-help/fix-adapter-reference/pl-calculation-algorithm/understanding-pl-calculations/?cn-reloaded=1
def get_pnlDf(strat: dict, candlesDf: pd.DataFrame) -> pd.DataFrame:
    """Returns two dataframes with pnl and trades based on the strategy parameters and candles dataframe"""
    buyStakeSize = strat['buyStakeSize']
    buyBp = strat['buyBp']
    buyCooldown = strat['buyCooldown']
    buyMaxContracts = strat['buyMaxContracts']
    sellStakeSize = strat['sellStakeSize']
    sellBp = strat['sellBp']
    sellCooldown = strat['sellCooldown']
    sellMinContracts = strat['sellMinContracts']
    lastBuyTradeTime = 0
    lastSellTradeTime = 0
    pnlRealized = 0
    pnlUnrealized = 0
    position = 0
    averageOpenPrice = 0
    pnlUnrealized = 0
    pnlTotal = 0
    comissionTotal = 0

    # create empty dataframes
    tradesDf = pd.DataFrame(columns=['updatetime', 'side', 'price',
                            'stakeSize', 'position', 'timeSinceLastTrade'])
    pnlDf = pd.DataFrame(columns=['updatetime', 'close', 'pnlRealized',
                         'pnlUnrealized', 'pnlTotal', 'comissionTotal', 'position', 'averageOpenPrice'])

    # for each candle calculate pnl and trades
    for index, row in candlesDf.iterrows():
        # of price is lower then ma, if position will not get grater then we specified and if cooldown is over
        if row['close'] * (1 + buyBp / 10000) < row['ma'] and \
                position + buyStakeSize <= buyMaxContracts and \
                row['updatetime'] - lastBuyTradeTime > buyCooldown * minute:
            # update lastBuyTradeTime and timeSinceLastBuyTrade
            timeSinceLastBuyTrade = (
                row['updatetime'] - lastBuyTradeTime) / minute
            lastBuyTradeTime = row['updatetime']
            # calculate comission
            comissionTotal += buyStakeSize * row['close'] * comissionRate
            # if position was >= 0 then new buy fills will not affect realized pnl
            # but will affect unrealized pnl and total pnl
            if position >= 0:
                averageOpenPrice = (averageOpenPrice * position + row['close'] * buyStakeSize) / \
                    (position + buyStakeSize)
                position += buyStakeSize
            # if position was < 0 then new buy fills will affect realized pnl, unrealized pnl and total pnl
            elif position < 0:
                pnlRealized += (row['close'] - averageOpenPrice) * \
                    min(buyStakeSize, -position)
                position += buyStakeSize
                # if position is still negative averageOpenPrice will not change
                # if position becomes 0 averageOpenPrice will be also 0
                # if position becomes positive averageOpenPrice will be equal to row['close']
                if position == 0:
                    averageOpenPrice = 0
                elif position > 0:
                    averageOpenPrice = row['close']
            tradesDf = tradesDf.append({
                'updatetime': row['updatetime'],
                'side': 'buy',
                'price': row['close'],
                'stakeSize': buyStakeSize,
                'position': position,
                'timeSinceLastTrade': timeSinceLastBuyTrade
            }, ignore_index=True)

        elif row['close'] * (1 - sellBp / 10000) >= row['ma'] and \
                position - sellStakeSize > sellMinContracts and \
                row['updatetime'] - lastSellTradeTime > sellCooldown * minute:
            timeSinceLastSellTrade = (
                row['updatetime'] - lastSellTradeTime) / minute
            lastSellTradeTime = row['updatetime']
            # calculate comission
            comissionTotal += sellStakeSize * row['close'] * comissionRate
            # if position < 0 then new sell fills will not affect realized pnl
            # but will affect unrealized pnl and total pnl
            if position <= 0:
                averageOpenPrice = (averageOpenPrice * abs(position) + row['close'] * sellStakeSize) / \
                    (abs(position) + sellStakeSize)
                position -= sellStakeSize

            # if position > 0 then new sell fills will affect realized pnl, unrealized pnl and total pnl
            elif position > 0:
                pnlRealized += (row['close'] - averageOpenPrice) * \
                    min(sellStakeSize, position)
                position -= sellStakeSize
                # if position is still positive averageOpenPrice will not change
                # if position becomes 0 averageOpenPrice will be also 0
                # if position becomes negative averageOpenPrice will be equal to row['close']
                if position == 0:
                    averageOpenPrice = 0
                elif position < 0:
                    averageOpenPrice = row['close']
            tradesDf = tradesDf.append({
                'updatetime': row['updatetime'],
                'side': 'sell',
                'price': row['close'],
                'stakeSize': sellStakeSize,
                'position': position,
                'timeSinceLastTrade': timeSinceLastSellTrade
            }, ignore_index=True)
        # update pnlUnrealized and pnlTotal for each candle
        pnlUnrealized = (row['close'] - averageOpenPrice) * position
        pnlTotal = pnlRealized + pnlUnrealized
        # add row to pnlDf
        pnlDf = pnlDf.append({
            'updatetime': row['updatetime'],
            'close': row['close'],
            'pnlRealized': pnlRealized,
            'pnlUnrealized': pnlUnrealized,
            'pnlTotal': pnlTotal,
            'comissionTotal': comissionTotal,
            'position': position,
            'averageOpenPrice': averageOpenPrice
        }, ignore_index=True)
    # add pnlFinal column which shows totalPnl with taking comission into account
    pnlDf['pnlFinal'] = pnlDf['pnlTotal'] - pnlDf['comissionTotal']
    return tradesDf, pnlDf


tradesDf, pnlDf = get_pnlDf(strat, candlesDf)


First let's have a look at tradesDF

In [33]:
tradesDf


,updatetime,side,price,stakeSize,position,timeSinceLastTrade
0,1.666959e+12,buy,1495.62,5,5,27782645.0
1,1.666959e+12,buy,1495.05,5,10,11.0
2,1.666960e+12,buy,1493.70,5,15,11.0
3,1.666960e+12,sell,1505.24,5,10,27782670.0
4,1.666961e+12,sell,1509.92,5,5,11.0
5,1.666962e+12,sell,1507.68,5,0,11.0
6,1.666962e+12,sell,1513.03,5,-5,11.0
7,1.666963e+12,sell,1519.32,5,-10,11.0
8,1.666966e+12,buy,1527.28,5,-5,101.0
9,1.666967e+12,buy,1527.29,5,0,11.0


We can see that position is always in our interval and timeSinceLastTrade is always grater than cooldown

Now let's have a look at pnlDf

In [34]:
pnlDf


,updatetime,close,pnlRealized,pnlUnrealized,pnlTotal,comissionTotal,position,averageOpenPrice,pnlFinal
0,1.666958e+12,1498.25,0.0,0.00,0.00,0.00000,0.0,0.000,0.00000
1,1.666958e+12,1498.78,0.0,0.00,0.00,0.00000,0.0,0.000,0.00000
2,1.666958e+12,1498.26,0.0,0.00,0.00,0.00000,0.0,0.000,0.00000
3,1.666958e+12,1498.24,0.0,0.00,0.00,0.00000,0.0,0.000,0.00000
4,1.666958e+12,1497.36,0.0,0.00,0.00,0.00000,0.0,0.000,0.00000
...,...,...,...,...,...,...,...,...,...
295,1.666976e+12,1552.40,423.2,-138.05,285.15,136.86355,-10.0,1538.595,148.28645
296,1.666976e+12,1553.00,423.2,-144.05,279.15,136.86355,-10.0,1538.595,142.28645
297,1.666976e+12,1552.79,423.2,-141.95,281.25,136.86355,-10.0,1538.595,144.38645
298,1.666976e+12,1563.57,423.2,-249.75,173.45,136.86355,-10.0,1538.595,36.58645


Now let's calculate final pnl with tradesDf and check that we get same numbers.

In [35]:
# initial pnl calculations from https://www.tradingtechnologies.com/xtrader-help/fix-adapter-reference/pl-calculation-algorithm/understanding-pl-calculations/?cn-reloaded=1
tradesDf['quoteQty'] = tradesDf['price'] * tradesDf['stakeSize']
totalBuyQty = tradesDf[tradesDf['side'] == 'buy']['stakeSize'].sum()
averageBuyPrice = tradesDf[tradesDf['side']
                           == 'buy']['quoteQty'].sum() / totalBuyQty
totalSellQty = tradesDf[tradesDf['side'] == 'sell']['stakeSize'].sum()
averageSellPrice = tradesDf[tradesDf['side'] ==
                            'sell']['quoteQty'].sum() / totalSellQty
pnlRealized = (averageSellPrice - averageBuyPrice) * \
    min(totalBuyQty, totalSellQty)

position = totalBuyQty - totalSellQty
if position > 0:
    averageOpenPrice = averageBuyPrice
elif position < 0:
    averageOpenPrice = averageSellPrice
else:
    averageOpenPrice = 0

pnlUnrealized = (candlesDf['close'].iloc[-1] - averageOpenPrice) * position
pnlTotal = pnlRealized + pnlUnrealized
comissionTotal = tradesDf['quoteQty'].sum() * comissionRate
pnlFinal = pnlTotal - comissionTotal

print('pnlRealized: ', pnlRealized)
print('pnlUnrealized: ', pnlUnrealized)
print('pnlTotal: ', pnlTotal)
print('comissionTotal: ', comissionTotal)
print('pnlFinal: ', pnlFinal)

tradesDf


pnlRealized:  341.8999999999869
pnlUnrealized:  -405.75000000000273
pnlTotal:  -63.850000000015825
comissionTotal:  136.86355
pnlFinal:  -200.71355000001583
